In [0]:
from pyspark.sql.functions import (
    col, count, avg, sum, min, max, 
    datediff, current_timestamp, month, year, desc, when, lit, concat_ws
)

# ---------------------------------------------------------
# 1. إعدادات المسارات (Configuration)
# ---------------------------------------------------------
storage_account_name = "hospitalstorge"
silver_static_path = f"abfss://silver@{storage_account_name}.dfs.core.windows.net/static_data"
gold_path = f"abfss://gold@{storage_account_name}.dfs.core.windows.net/kpi_data"

# إعدادات الـ Hive Metastore (الحل المستقر)
catalog_name = "hive_metastore"
db_name = "gold_analytics"

# إنشاء قاعدة البيانات
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_name}.{db_name}")

print("🚀 Starting Gold Layer Processing...")

# ---------------------------------------------------------
# 2. قراءة بيانات Silver (Extraction)
# ---------------------------------------------------------
print("Reading Dimensions from Silver...")
df_hospitals = spark.read.format("delta").load(f"{silver_static_path}/silver_hospitals")
df_staff = spark.read.format("delta").load(f"{silver_static_path}/silver_staff")

print("Reading Patients Dimension...")
try:
    df_patients = spark.read.table("silver_data.dim_patients")
except:
    print("⚠️ Reading dim_patients from Checkpoint path...")
    df_patients = spark.read.format("delta").load(f"abfss://silver@{storage_account_name}.dfs.core.windows.net/Checkpoints/dim_patients")

print("Reading Fact Table...")
try:
    df_admissions = spark.read.table("silver_data.fact_admissions")
except:
    print("⚠️ Reading fact_admissions from Checkpoint path...")
    df_admissions = spark.read.format("delta").load(f"abfss://silver@{storage_account_name}.dfs.core.windows.net/Checkpoints/fact_admissions")

# ---------------------------------------------------------
# 3. التحويلات وحساب المقاييس (Transformations & KPIs)
# ---------------------------------------------------------

# حساب مدة الإقامة
df_fact_enriched = df_admissions.withColumn(
    "length_of_stay_days", 
    datediff(col("discharge_time"), col("admission_time"))
)

# --- KPI 1: ملخص أداء المستشفيات ---
print("Calculating Hospital KPIs...")
df_gold_hospitals_kpi = df_fact_enriched.join(df_hospitals, "hospital_id") \
    .groupBy("hospital_name", "city", "department") \
    .agg(
        count("record_id").alias("total_patients"),
        avg("length_of_stay_days").alias("avg_stay_days"),
        sum(when(col("severity_level") == "Critical", 1).otherwise(0)).alias("critical_cases"),
        sum(when(col("ICU_admission") == True, 1).otherwise(0)).alias("icu_admissions")
    ) \
    .withColumn("last_refreshed", current_timestamp())

# --- KPI 2: تحليل الأمراض (Disease Analysis) - [تم التصحيح هنا] ---
print("Calculating Disease Trends...")
# نستخدم محافظة المريض (df_patients.governorate)
df_gold_disease_trends = df_fact_enriched \
    .join(df_hospitals, "hospital_id") \
    .join(df_patients, "patient_id") \
    .groupBy(df_patients["governorate"], "diagnosis", "gender") \
    .agg(
        count("record_id").alias("case_count"),
        avg("length_of_stay_days").alias("avg_treatment_days")
    ) \
    .orderBy(desc("case_count"))

# --- KPI 3: أداء الأطباء ---
print("Calculating Doctor Workload...")
df_gold_doctor_stats = df_fact_enriched \
    .join(df_staff, df_fact_enriched.doctor_id == df_staff.staff_id, "inner") \
    .join(df_hospitals, df_fact_enriched.hospital_id == df_hospitals.hospital_id, "inner") \
    .withColumn("doctor_full_name", concat_ws(" ", df_staff["first_name"], df_staff["last_name"])) \
    .groupBy("doctor_full_name", "specialization", "hospital_name") \
    .agg(
        count("record_id").alias("patients_treated"),
        count(when(col("severity_level") == "Critical", 1)).alias("critical_cases_handled")
    )

# ---------------------------------------------------------
# 4. الحفظ في Gold (Loading)
# ---------------------------------------------------------
def save_to_gold(df, table_name):
    full_path = f"{gold_path}/{table_name}"
    
    # 1. حفظ ملفات البيانات
    df.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .save(full_path)
    
    # 2. تسجيل الجدول في hive_metastore
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {catalog_name}.{db_name}.{table_name}
        USING DELTA LOCATION '{full_path}'
    """)
    print(f"✅ Saved and Registered: {catalog_name}.{db_name}.{table_name}")

# تنفيذ الحفظ
save_to_gold(df_gold_hospitals_kpi, "hospital_performance_kpi")
save_to_gold(df_gold_disease_trends, "disease_geography_trends")
save_to_gold(df_gold_doctor_stats, "doctor_performance_kpi")

print("\n🎉 Gold Layer processing finished successfully!")

🚀 Starting Gold Layer Processing...
Reading Dimensions from Silver...
Reading Patients Dimension...
Reading Fact Table...
Calculating Hospital KPIs...
Calculating Disease Trends...
Calculating Doctor Workload...
✅ Saved and Registered: hive_metastore.gold_analytics.hospital_performance_kpi
✅ Saved and Registered: hive_metastore.gold_analytics.disease_geography_trends
✅ Saved and Registered: hive_metastore.gold_analytics.doctor_performance_kpi

🎉 Gold Layer processing finished successfully!
